In [ ]:
import nltk
from nltk import word_tokenize
from nltk.corpus import brown as cb
from nltk.corpus import treebank as tb
import pprint
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import Pipeline

In [ ]:
raw_text = nltk.Text(cb.words('ca01'))
print (raw_text)

In [ ]:
print (cb.words()[0:20])

['The', 'Fulton', 'County', 'Grand', 'Jury', 'said', 'Friday', 'an', 'investigation', 'of', "Atlanta's", 'recent', 'primary', 'election', 'produced', '``', 'no', 'evidence', "''", 'that']


In [ ]:
print (cb.tagged_words()[0:10])

[('The', 'AT'), ('Fulton', 'NP-TL'), ('County', 'NN-TL'), ('Grand', 'JJ-TL'), ('Jury', 'NN-TL'), ('said', 'VBD'), ('Friday', 'NR'), ('an', 'AT'), ('investigation', 'NN'), ('of', 'IN')]


In [ ]:
tagged_sentences_brown_corpus = nltk.corpus.brown.tagged_sents()
pprint.pprint(tagged_sentences_brown_corpus[0])


[('The', 'AT'),
 ('Fulton', 'NP-TL'),
 ('County', 'NN-TL'),
 ('Grand', 'JJ-TL'),
 ('Jury', 'NN-TL'),
 ('said', 'VBD'),
 ('Friday', 'NR'),
 ('an', 'AT'),
 ('investigation', 'NN'),
 ('of', 'IN'),
 ("Atlanta's", 'NP$'),
 ('recent', 'JJ'),
 ('primary', 'NN'),
 ('election', 'NN'),
 ('produced', 'VBD'),
 ('``', '``'),
 ('no', 'AT'),
 ('evidence', 'NN'),
 ("''", "''"),
 ('that', 'CS'),
 ('any', 'DTI'),
 ('irregularities', 'NNS'),
 ('took', 'VBD'),
 ('place', 'NN'),
 ('.', '.')]


In [ ]:
raw_text = nltk.Text(tb.words()[0:10])
print (raw_text)

<Text: Pierre Vinken , 61 years old , will...>


In [ ]:
print (tb.words()[0:10])

['Pierre', 'Vinken', ',', '61', 'years', 'old', ',', 'will', 'join', 'the']


In [ ]:
tagged_sentences_treebank_corpus = nltk.corpus.treebank.tagged_sents()
pprint.pprint (tagged_sentences_treebank_corpus[0])

[('Pierre', 'NNP'),
 ('Vinken', 'NNP'),
 (',', ','),
 ('61', 'CD'),
 ('years', 'NNS'),
 ('old', 'JJ'),
 (',', ','),
 ('will', 'MD'),
 ('join', 'VB'),
 ('the', 'DT'),
 ('board', 'NN'),
 ('as', 'IN'),
 ('a', 'DT'),
 ('nonexecutive', 'JJ'),
 ('director', 'NN'),
 ('Nov.', 'NNP'),
 ('29', 'CD'),
 ('.', '.')]


<h4 align="center"> We will be using Treebank corpus to build our own POS tagger </h4>

In [ ]:
print ("Tagged sentences: ", len(tagged_sentences_treebank_corpus))

Tagged sentences:  3914


In [ ]:
print ("Tagged words:", len(nltk.corpus.treebank.tagged_words()))

Tagged words: 100676


###  Generate features

#### Function for generating features form tagged corpus

In [ ]:
def features(sentence, index):
    # "sentence: [w1, w2, ...], index: the index of the word"
    return {
    'word': sentence[index],
    'is_first': index == 0,
    'is_last': index == len(sentence) - 1,
    'is_capitalized': sentence[index][0].upper() == sentence[index][0],
    'is_all_caps': sentence[index].upper() == sentence[index],
    'is_all_lower': sentence[index].lower() == sentence[index],
    'prefix-1': sentence[index][0],
    'prefix-2': sentence[index][:2],
    'prefix-3': sentence[index][:3],
    'suffix-1': sentence[index][-1],
    'suffix-2': sentence[index][-2:],
    'suffix-3': sentence[index][-3:],
    'prev_word': '' if index == 0 else sentence[index - 1],
    'next_word': '' if index == len(sentence) - 1 else sentence[index + 1],
    'has_hyphen': '-' in sentence[index],
    'is_numeric': sentence[index].isdigit(),
    'capitals_inside': sentence[index][1:].lower() != sentence[index][1:]
    }

In [ ]:
pprint.pprint(features(['This', 'is', 'a', 'sentence'], 0))

{'capitals_inside': False,
 'has_hyphen': False,
 'is_all_caps': False,
 'is_all_lower': False,
 'is_capitalized': True,
 'is_first': True,
 'is_last': False,
 'is_numeric': False,
 'next_word': 'is',
 'prefix-1': 'T',
 'prefix-2': 'Th',
 'prefix-3': 'Thi',
 'prev_word': '',
 'suffix-1': 's',
 'suffix-2': 'is',
 'suffix-3': 'his',
 'word': 'This'}


In [ ]:
def untag(tagged_sentence):
    return [w for w, t in tagged_sentence]

In [ ]:
def transform_to_dataset(tagged_sentences):
    X, y = [], []
    for tagged in tagged_sentences:
        for index in range(len(tagged)):
            X.append(features(untag(tagged), index))
            y.append(tagged[index][1])
            #pprint.pprint(" original word: "+ str(tagged) + " Word: "+ str(untag(tagged))+ "Y: " + y[index])
    return X, y

In [ ]:
cutoff = int(.75 * len(tagged_sentences_treebank_corpus))
training_sentences = tagged_sentences_treebank_corpus[:cutoff]
test_sentences = tagged_sentences_treebank_corpus[cutoff:]

In [ ]:
print (len(training_sentences))
print (len(test_sentences))

2935
979


In [ ]:
X, y = transform_to_dataset(training_sentences)

In [ ]:
print(len(X))
print(len(y))

75784
75784


In [ ]:
clf = Pipeline([
    ('vectorizer', DictVectorizer(sparse=False)),
    ('classifier', DecisionTreeClassifier(criterion='entropy'))
])

In [ ]:
# Use only the first 10K samples if you're running it multiple times. It takes a fair bit :)
clf.fit(X[:20000],y[:20000])

Pipeline(memory=None,
     steps=[('vectorizer', DictVectorizer(dtype=<class 'numpy.float64'>, separator='=', sort=True,
        sparse=False)), ('classifier', DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'))])

In [ ]:
X_test, y_test = transform_to_dataset(test_sentences)

In [ ]:
print ("Accuracy:{:.3%}".format(clf.score(X_test, y_test)))

Accuracy:91.387%


In [ ]:
def pos_tag(sentence):
    tagged_sentence = []
    tags = clf.predict([features(sentence, index) for index in range(len(sentence))])
    return zip(sentence, tags)


In [ ]:
POS_list = list(pos_tag(word_tokenize('This is my friend, John.')))
for t in POS_list:
    print(u"{:<16}{:>2}".format(str(t[0]),str(t[1])))

This            DT
is              VBZ
my              PRP$
friend          NN
,                ,
John            NNP
.                .


In [ ]:
POS_list = list(pos_tag(word_tokenize("We will meet at eight o'clock on Thursday morning.")))
for t in POS_list:
    print(u"{:<16}{:>2}".format(str(t[0]),str(t[1])))

We              IN
will            MD
meet            VB
at              IN
eight           NN
o'clock         NN
on              IN
Thursday        DT
morning         NN
.                .


In [ ]:
POS_list = list(pos_tag(word_tokenize('Alexander, the great...!')))
for t in POS_list:
    print(u"{:<16}{:>2}".format(str(t[0]),str(t[1])))

Alexander       IN
,                ,
the             DT
great           NN
...              :
!               CD


In [ ]:
POS_list = list(pos_tag(word_tokenize('Alexander the Great, was a king of the ancient Greek kingdom of Macedon.')))
for t in POS_list:
    print(u"{:<16}{:>2}".format(str(t[0]),str(t[1])))


Alexander       IN
the             DT
Great           NNP
,                ,
was             VBD
a               DT
king            NN
of              IN
the             DT
ancient         DT
Greek           NNP
kingdom         NN
of              IN
Macedon         NNP
.                .
